In [1]:
!pip install spyne


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [2]:
!pip install twisted


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [6]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [10]:
conda install -c conda-forge spacy-model-fr_core_news_sm

done
Solving environment: done

## Package Plan ##

  environment location: /Users/baldita/anaconda3

  added / updated specs:
    - spacy-model-fr_core_news_sm


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    spacy-model-fr_core_news_sm-3.5.0|     pyhd8ed1ab_0        14.9 MB  conda-forge
    ------------------------------------------------------------
                                           Total:        14.9 MB

The following NEW packages will be INSTALLED:

  spacy-model-fr_co~ conda-forge/noarch::spacy-model-fr_core_news_sm-3.5.0-pyhd8ed1ab_0 



                                                                                
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [1]:
import re
import spacy
import json

In [2]:
def nlp_extract(texte):
    # Charger le modèle spaCy
    nlp = spacy.load("fr_core_news_sm")

    # Traitement du texte avec spaCy
    doc = nlp(texte)

    # Initialisation d'un dictionnaire pour stocker les informations extraites
    resultat = {
        "prenom": "",
        "nom": "",
        "montant": "",
        "logement": ""
    }

    # Parcourir les tokens du texte
    for token in doc:
        if token.ent_type_ == "PER":  # Entité de type personne
            if not resultat["prenom"]:
                resultat["prenom"] = token.text
            else:
                resultat["nom"] = token.text

    # Utilisation d'une expression régulière pour extraire le montant
    montant_pattern = r'\d{1,10}(?:[.,]\d{1,2})? euros'
    montant_match = re.search(montant_pattern, texte)
    if montant_match:
        resultat["montant"] = montant_match.group(0)

    # Recherche du type de logement
    logement_idx = texte.find("pour un logement")
    if logement_idx != -1:
        resultat["logement"] = texte[logement_idx + len("pour un logement"):].strip()

    # Convertir le dictionnaire en JSON
    resultat_json = json.dumps(resultat, ensure_ascii=False, indent=4)

    # Afficher le résultat au format JSON
    return resultat_json



In [3]:
import logging
logging.basicConfig(level=logging.DEBUG)
import sys
from spyne import Application, rpc, ServiceBase, Unicode, Integer, Iterable
from spyne.protocol.soap import Soap11
from spyne.server.wsgi import WsgiApplication
from spyne.util.wsgi_wrapper import run_twisted
from xml.sax.saxutils import escape

DEBUG:spyne.model._base:	string: setting pattern='[a-fA-F0-9]{8}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{12}'
DEBUG:spyne.model._base:	string: setting pattern='[a-fA-F0-9]{8}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{12}'
DEBUG:spyne.model._base:	string: setting unicode_pattern='\\w+(\\.\\w+)*'
DEBUG:spyne.model._base:	string: setting max_len=2048
DEBUG:spyne.model._base:	string: setting unicode_pattern='\\w+(\\.\\w+)*'
DEBUG:spyne.model._base:	string: setting max_len=65536
DEBUG:spyne.model._base:	string: setting unicode_pattern="(application|audio|font|example|image|message|model|multipart|text|video|x-(?:[0-9A-Za-z!#$%&'*+.^_`|~-]+))/([0-9A-Za-z!#$%&'*+.^_`|~-]+)"
DEBUG:spyne.model._base:	string: setting unicode_pattern="[0-9A-Za-z!#$%&'*+.^_`|~-]+/([0-9A-Za-z!#$%&'*+.^_`|~-]+)"
DEBUG:spyne.model._base:	string: setting unicode_pattern='(application|audio|font|example|image|message|model|multipart|text|video|x-(?:[0-9A-Za-z!#$%&\'*+.^_`|~-]+))/([0-9A-Z

DEBUG:spyne.model._base:	short: setting min_occurs=1
DEBUG:spyne.model._base:	short: setting nillable=False
DEBUG:spyne.model._base:	short: setting max_str_len=Decimal('Infinity')
DEBUG:spyne.model._base:	byte: setting _explicit_type_name=True because we have 'type_name'
DEBUG:spyne.model._base:	byte: setting min_occurs=1
DEBUG:spyne.model._base:	byte: setting nillable=False
DEBUG:spyne.model._base:	byte: setting max_str_len=Decimal('Infinity')
DEBUG:spyne.model._base:	nonNegativeInteger: setting _explicit_type_name=True because we have 'type_name'
DEBUG:spyne.model._base:	nonNegativeInteger: setting min_occurs=1
DEBUG:spyne.model._base:	nonNegativeInteger: setting nillable=False
DEBUG:spyne.model._base:	nonNegativeInteger: setting max_str_len=Decimal('Infinity')
DEBUG:spyne.model._base:	unsignedLong: setting _explicit_type_name=True because we have 'type_name'
DEBUG:spyne.model._base:	unsignedLong: setting min_occurs=1
DEBUG:spyne.model._base:	unsignedLong: setting nillable=False
DEBU

In [ ]:
class extractionInformationService(ServiceBase):
    @rpc(Unicode,_returns=Iterable(Unicode))
    def extraire_information(ctx, demande):
        infos = escape(nlp_extract(demande))
        yield f"Voici les infos extraites de la demande, {infos}"

application = Application([extractionInformationService],
                        tns='spyne.examples.hello',
                        in_protocol=Soap11(validator='lxml'),
                        out_protocol=Soap11()
                    )


if __name__ == '__main__':

    wsgi_app = WsgiApplication(application)


    twisted_apps = [
        (wsgi_app, b'extractionInformationService'),
    ]

    sys.exit(run_twisted(twisted_apps,8008))

DEBUG:spyne.model.complex:Got bases for Iterable from orig: (<class 'spyne.model.complex.Array'>,)
DEBUG:spyne.model._base:	string: setting max_occurs=Decimal('Infinity')
DEBUG:spyne.model.complex:Got bases for extraire_information from meta: (<class 'spyne.model.complex.ComplexModel'>,)
DEBUG:spyne.model.complex:Got bases for extraire_informationResponse from meta: (<class 'spyne.model.complex.ComplexModel'>,)
INFO:spyne.application:Initializing application {spyne.examples.hello}Application...
DEBUG:spyne.interface._base:populating __main__.extractionInformationService types...
DEBUG:spyne.interface._base:  enumerating classes for method 'extraire_information'
DEBUG:spyne.interface._base:    adding class "<class 'spyne.model.complex.extraire_information'>" for '{spyne.examples.hello}extraire_information'
DEBUG:spyne.interface._base:    adding extraire_information.demande = <class 'spyne.model.primitive.string.Unicode'>
DEBUG:spyne.interface._base:    adding class "<class 'spyne.model.

DEBUG:spyne.protocol._base:<spyne.protocol.soap.soap11.Soap11 object at 0x12d166a90> attrcache size: 4
DEBUG:spyne.protocol._base:<spyne.protocol.soap.soap11.Soap11 object at 0x12d166a90> attrcache size: 4
DEBUG:spyne:gc.collect() took around 250ms.
DEBUG:spyne.server.wsgi:Add http header 'host' = ['0.0.0.0:8008']
DEBUG:spyne.server.wsgi:Add http header 'user_agent' = ['python-requests/2.31.0']
DEBUG:spyne.server.wsgi:Add http header 'accept_encoding' = ['gzip, deflate, br']
DEBUG:spyne.server.wsgi:Add http header 'accept' = ['*/*']
DEBUG:spyne.server.wsgi:Add http header 'connection' = ['keep-alive']
DEBUG:spyne.server.wsgi:Add http header 'content_type' = ['application/soap+xml; charset=utf-8']
DEBUG:spyne.server.wsgi:Add http header 'content_length' = ['455']
DEBUG:spyne.protocol.xml:Validated ? True
DEBUG:spyne.protocol._base:<spyne.protocol.soap.soap11.Soap11 object at 0x12d7d2050> attrcache size: 2
DEBUG:spyne.protocol._base:<spyne.protocol.soap.soap11.Soap11 object at 0x12d7d205

In [26]:
v=nlp_extract("Je m'appelle Jean Dupont et je souhaite emprunter 200000 euros pour un logement de 300 m2 à Paris.")

In [28]:
print(v)

{
    "prenom": "Jean",
    "nom": "Dupont",
    "montant": "200000 euros",
    "logement": "de 300 m2 à Paris."
}
